<a href="https://colab.research.google.com/github/bbrady15/poseidonseye/blob/main/NRL_Final_Ship_Image_Detection_%26_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Install Requirements
* Installs YOLOv5 and assossociated required packages using requirements.txt
* Installs Roboflow to import images from dataset

In [1]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 12576, done.
remote: Total 12576 (delta 0), reused 0 (delta 0), pack-reused 12576
Receiving objects: 100% (12576/12576), 12.47 MiB | 35.47 MiB/s, done.
Resolving deltas: 100% (8621/8621), done.
/content/yolov5
     |████████████████████████████████| 596 kB 9.7 MB/s 
     |████████████████████████████████| 145 kB 7.8 MB/s 
     |████████████████████████████████| 178 kB 67.5 MB/s 
     |████████████████████████████████| 1.1 MB 65.4 MB/s 
     |████████████████████████████████| 67 kB 7.7 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |████████████████████████████████| 138 kB 91.6 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
Setup complete. Using torch 1.12.0+cu113 (Tesla P100-PCIE-16GB)


# Step 2: Assemble Our Dataset
Creates directory variable for storing dataset and then imports dataset from Roboflow



In [2]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "../content/datasets"

In [3]:
#after following the link above, recieve python code with these fields filled in
from roboflow import Roboflow
rf = Roboflow(api_key="nHrF2qASypVi7bc1T3Sa")
project = rf.workspace("usna-dcxiv").project("ships-4od7b")
dataset = project.version(14).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to ../content/datasets/Ships-14 in yolov5pytorch:: 100%|██████████| 17048/17048 [00:02<00:00, 7805.33it/s]


# Step 3: Train Our Custom YOLOv5 model

Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** Our dataset locaiton is saved in the `dataset.location`
- **weights:** specify a path to weights to start transfer learning from. Here we choose the generic COCO pretrained checkpoint. (usually best.pt if already trained once)
- **cache:** cache images for faster training

In [ ]:
#!python train.py --img 416 --batch 128 --epochs 500 --data {dataset.location}/data.yaml --cfg /content/yolov5/models/yolov5s.yaml --cache
!python train.py --img 416 --batch 128 --epochs 250 --cfg /content/yolov5/models/yolov5s.yaml --data {dataset.location}/data.yaml --cache

train: weights=yolov5s.pt, cfg=/content/yolov5/models/yolov5s.yaml, data=/content/content/datasets/Ships-14/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=250, batch_size=128, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-379-gf5335f2 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_p

# Step 4: Evaluate Custom YOLOv5 Detector Performance
Training losses and performance metrics are saved to Tensorboard and also to a logfile.

If you are new to these metrics, the one you want to focus on is `mAP_0.5` - learn more about mean average precision [here](https://blog.roboflow.com/mean-average-precision/).

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

#Step 5: Run Inference  With Trained Weights
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.
- **source:** path to video

In [ ]:
# !python detect.py --weights runs/train/exp/weights/best.pt --img 416 --conf 0.1 --source {dataset.location}/test/images
!python detect.py --weights /content/yolov5/runs/train/exp4/weights/best.pt --img 416 --conf 0.1 --source /content/smoothtimelapse.mp4

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 416 --conf 0.1 --source /content/vidtest1.mp4

In [ ]:
!pip install sahi

In [ ]:
!sahi predict --model_path /content/yolov5/runs/train/exp4/weights/best.pt --model_type yolov5 --source /content/smoothtimelapse.mp4